# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645-675) underperformed compared to schools with smaller budgets (<\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.dtypes

Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
School ID         int64
type             object
size              int64
budget            int64
dtype: object

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [30]:
# Calculate the 8 requested quantities
total_schools = school_data_complete['School ID'].nunique()
total_students = school_data_complete['Student ID'].nunique()
total_budget = school_data_complete.groupby("School ID").budget.first().sum()
average_math_score = school_data_complete.math_score.mean()
average_reading_score = school_data_complete.reading_score.mean()
overall_passing_rate = 0.5 * average_math_score + 0.5 * average_reading_score
percent_passing_math = school_data_complete.math_score.loc[school_data_complete['math_score'] > 70].count() / total_students * 100
percent_passing_reading = school_data_complete.reading_score.loc[school_data_complete['reading_score'] > 70].count() / total_students * 100
# Convert to DataFrame 
summary_table = pd.DataFrame({'Total Schools':total_schools,'Total Students':total_students,'Total Budget':total_budget\
                             ,'Average Math Score':average_math_score, 'Average Reading Score':average_reading_score\
                             ,'% Passing Math':percent_passing_math,'% Passing Reading':percent_passing_reading\
                             ,"% Overall Passing Rate":overall_passing_rate},index=np.arange(1))
# Clean up formatting and display
summary_table['Total Students'] = summary_table['Total Students'].map('{:,}'.format)
summary_table['Total Budget'] = summary_table['Total Budget'].map('${:,}'.format)
summary_table.style.hide_index()

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
15,"39,170","$24,649,428",78.9854,81.8778,72.3921,82.9717,80.4316


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [62]:
# Use a bulk groupby("school_name") to get name, type, total students, total budget, average math score, average reading score
school_names1_df = school_data_complete.groupby("school_name").agg({'type':'first','Student ID':'nunique','budget':'first'\
                                                               ,'math_score':'mean','reading_score':'mean'}).reset_index()
# Fix the multi-index, the column names will be 'type','Student ID','budget','math_score','reading_score'
school_names1_df.columns = school_names_df.columns.get_level_values(0)
# Add a column for budget per pupil
school_names1_df['Budget per Student'] = school_names1_df['budget'].astype(np.float) / school_names1_df['Student ID'].astype(np.float)
# Create new DataFrames with groupby("school_name") for percent passing math and reading
# This is done by first creating two seprate DataFrames that holds the counts of passing students ...
school_names_mp1_df = school_data_complete[school_data_complete['math_score'] > 70].groupby("school_name")[['math_score']].count().reset_index() 
school_names_rp1_df = school_data_complete[school_data_complete['reading_score'] > 70].groupby("school_name")[['reading_score']].count().reset_index()
# Then adding columns to capture the percentage passing to each data frame
school_names_mp1_df['% Passing Math'] = school_names_mp1_df['math_score'] / school_names1_df['Student ID'] * 100
school_names_rp1_df['% Passing Reading'] = school_names_rp1_df['reading_score'] / school_names1_df['Student ID'] * 100
# Next, these new columns will be merged into a single DataFrame
school_names_merge_mr = school_names_mp1_df.merge(school_names_rp1_df,on='school_name',how='outer')
# Add the final column for the overall passing rate
school_names_merge_mr['Overall Passing Rate'] = 0.5 * (school_names_merge_mr['% Passing Math'] + school_names_merge_mr['% Passing Reading'])
# Then trimming away the columns we no longer need to use & preparing to merge with school_names1_df
school_names2_df = school_names_merge_mr.drop(['math_score','reading_score'], axis=1)
# Merge DataFrames school_names1_df and school_names2_df to create the final table
school_names1_df = school_names1_df.merge(school_names2_df,on='school_name',how='outer')
# Lastly, clean up the column names to be more meaningful and fix formatting
school_names1_df = school_names1_df.rename(columns = {'type':'School Type','Student ID':'Total Students','budget':'Total Budget', \
                                    'math_score':'Average Math Score','reading_score':'Average Reading Score'})
school_names1_df['Total Students'] = school_names1_df['Total Students'].map('{:,}'.format)
school_names1_df['Total Budget'] = school_names1_df['Total Budget'].map('${:,}'.format)
school_names1_df['Budget per Student'] = school_names1_df['Budget per Student'].map('${:,.2f}'.format)
school_names1_df

,school_name,School Type,Total Students,Total Budget,Average Math Score,Average Reading Score,Budget per Student,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,"4,976","$3,124,928",77.048432,81.033963,$628.00,64.630225,79.300643,71.965434
1,Cabrera High School,Charter,"1,858","$1,081,356",83.061895,83.975780,$582.00,89.558665,93.864370,91.711518
2,Figueroa High School,District,"2,949","$1,884,411",76.711767,81.158020,$639.00,63.750424,78.433367,71.091896
3,Ford High School,District,"2,739","$1,763,916",77.102592,80.746258,$644.00,65.753925,77.510040,71.631982
4,Griffin High School,Charter,"1,468","$917,500",83.351499,83.816757,$625.00,89.713896,93.392371,91.553134
5,Hernandez High School,District,"4,635","$3,022,020",77.289752,80.934412,$652.00,64.746494,78.187702,71.467098
6,Holden High School,Charter,427,"$248,087",83.803279,83.814988,$581.00,90.632319,92.740047,91.686183
7,Huang High School,District,"2,917","$1,910,635",76.629414,81.182722,$655.00,63.318478,78.813850,71.066164
8,Johnson High School,District,"4,761","$3,094,650",77.072464,80.966394,$650.00,63.852132,78.281874,71.067003
9,Pena High School,Charter,962,"$585,858",83.839917,84.044699,$609.00,91.683992,92.203742,91.943867


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [64]:
school_names1_df.sort_values('Overall Passing Rate',ascending=False).head(5)

,school_name,School Type,Total Students,Total Budget,Average Math Score,Average Reading Score,Budget per Student,% Passing Math,% Passing Reading,Overall Passing Rate
13,Wilson High School,Charter,"2,283","$1,319,574",83.274201,83.989488,$578.00,90.932983,93.254490,92.093736
9,Pena High School,Charter,962,"$585,858",83.839917,84.044699,$609.00,91.683992,92.203742,91.943867
14,Wright High School,Charter,"1,800","$1,049,400",83.682222,83.955000,$583.00,90.277778,93.444444,91.861111
1,Cabrera High School,Charter,"1,858","$1,081,356",83.061895,83.975780,$582.00,89.558665,93.864370,91.711518
6,Holden High School,Charter,427,"$248,087",83.803279,83.814988,$581.00,90.632319,92.740047,91.686183


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [17]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
